# Working with multidimensional spatial data in Python
# lesson 2.1 - Processing imaging spectroscopy data at scale
_Glenn Moncrieff_  
[github.com/GMoncrieff](github.com/GMoncrieff)  
[@glennwithtwons](https://twitter.com/glennwithtwons)

Now that we are familiar with the main packages and concepts related to scalable processing of geospatial data in python we will start analysing some real imaging spectroscopy data. We will train a model that will be used to unmix the endmembers in a imaging spectroscopy data cube. Our goal is to map the distribution of invasive alien pine tress in the Cape florsitic region of South Africa using data from [The Earth Surface Mineral Dust Source Investigation (EMIT)](https://earth.jpl.nasa.gov/emit/).  
  
  
The unmxing approach we will use is regression-based unmixing. This involves creating synthetic mixtures from pure endmembers with known mixing proportions. We then used these synthetic mixutres to train machine learning models. See [Okunjeni et al 2013 RSE](https://www.sciencedirect.com/science/article/abs/pii/S0034425713002009) for more background.    
  
  
In this first notebook we access the EMIT data, create the synthetic mixtures, then train and save our model.

## 1. Setup access

### Authenticate with NASA Earthdata portal
Earthaccess is a python library to search, download or stream NASA Earth science data. You will also need an account on NASA's Earthdata data portal
https://search.earthdata.nasa.gov/

In [ ]:
import earthaccess
auth = earthaccess.login(persist=True)

In [ ]:
#The old way
#from utils.s3_access import write_creds
#write_creds()

## 2. Load libraries

> a number of utility functions that I have written/copied are saved as `.py` files in the `utils` folder. We load these using `from utils.filename import functionname`. You can browse the files to understand the code, but I have moved the functions out of this notebook to reduce the cognitive load and keep the code neat.

In [ ]:
#imports
import s3fs
import xarray as xr
import hvplot.xarray
import holoviews as hv
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import netCDF4 as nc
import json
import geopandas as gpd

#ml modules
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

#ml models
from xgboost import XGBRegressor

#our modules
from utils.emit_tools import emit_xarray, quality_mask, gamma_adjust
from utils.synthmix import extract_points, mix_fast


## 3. Accessing and finding files

### Accessing data on AWS S3

The emit files are stored on Amazon S3 object storage. We will access these directly without needing to download them as we can stream s3 files. Also, because we are using xarray with dask and our data is stored in a chunked file format `netcdf`, we only need to stream the chunks we are currently working on and not the entire file at once.
  
The best way to access the data is on a virtual machine located in `us-west-2`. This is where the data is located and this mean that will get the fastest reading and writing of data possible, and all read/writes will be free. If your machine is not in `us-west-2` or you are on a local machine, I do not recommend using S3, as it might be slower and there will be costs involved.

### Finding data

earthaccess has really convineint methods for searching and loading nasa data over S3 using `earthaccess.search_data()`. Here is an example:
```
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(19.03711,-33.94747),
    temporal=('2023-01-18','2023-01-24'),
    count=2
)


you can then load the results into an xarray
```
files = earthaccess.open(results)
ds = xr.open_mfdataset(files)


if you do not want to stream the data from S3 you can download the data directly using:
```
files = earthaccess.download(results, "./local_folder")


for more info on using earthaccess see the [docs](https://nsidc.github.io/earthaccess/)

I have already saved the links to the EMIT scenes we will analyses on S3 (you can find scenes interactively on [the earthdata serarch page](https://search.earthdata.nasa.gov/search) , so we will not use the `earthacess` package to find data.

In [ ]:
#these are our files - the s3 links
f_url = ['s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230119T114247_2301907_005/EMIT_L2A_RFL_001_20230119T114247_2301907_005.nc',
         's3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230123T100615_2302306_006/EMIT_L2A_RFL_001_20230123T100615_2302306_006.nc']
f_mask_url = ['s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230119T114247_2301907_005/EMIT_L2A_MASK_001_20230119T114247_2301907_005.nc',
            's3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230123T100615_2302306_006/EMIT_L2A_MASK_001_20230123T100615_2302306_006.nc']

## 4. Open the datset

### If you are in us-west-2: Authenticate with s3 and stream data

In [ ]:
#load s3 credentials
from utils.s3_access import get_temp_creds
temp_creds_req = get_temp_creds()

In [ ]:
# Pass Authentication to s3fs
fs_s3 = s3fs.S3FileSystem(anon=False, 
                          key=temp_creds_req['accessKeyId'], 
                          secret=temp_creds_req['secretAccessKey'], 
                          token=temp_creds_req['sessionToken'])

#### Link to the s3 file and have a quick look with xarray

In [ ]:
# Open s3 url
fp = fs_s3.open(f_url[0], mode='rb')
fp_mask = fs_s3.open(f_mask_url[0], mode='rb')

# Open dataset with xarray
ds = xr.open_dataset(fp,engine='h5netcdf',chunks='auto')
ds

### If you are not in us-west-2: Download locally

In [ ]:
#get the files we want
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(19.03711,-33.94747),
    temporal=('2023-01-18','2023-01-24'),
    count=1
)
#download them
files = earthaccess.download(results, "data/downloads")

In [ ]:
# Open dataset with xarray
fp = 'data/downloads/EMIT_L2A_RFL_001_20230119T114247_2301907_005.nc'
fp_mask = 'data/downloads/EMIT_L2A_MASK_001_20230119T114247_2301907_005.nc'

ds = xr.open_dataset(fp,engine='h5netcdf',chunks='auto')
ds

### Load the mask
The EMIT data is composed of 3 files, the actual data, a mask that tells us about various quality issues with the data, and the reflectance uncertainties (we will ignore the uncertainty for now). We can selecting which quality filter (mask) to use. Here we used the aggregate filter - the sum of all filters to be very strict about what data we include. The `quality_mask` function provided by the EMIT team reads and processes this file for us.

In [ ]:
flags=[7]
mask = quality_mask(fp_mask,flags)

### Load the data
As you saw the raw data is the unprojected, unmasked focal plane array and does not include much metadata for us to tell what is going on.  The `emit_array` function provided by the EMIT team reads and processes this file for us. It can orthorectify the data and apply the mask too if we want.

> Warning: in order to orthorectify the data, the entire array must be loaded into memory. This means you need a decent amount of RAM or the operation will fail. I found that 16GB is the minimum needed.


In [ ]:
ds = emit_xarray(fp, ortho=True,qmask=mask)
ds

Now that we have orthorectified the data we can chunk it to make later processes more efficient

In [ ]:
ds = ds.chunk({'latitude':-1,'longitude':-1,'wavelengths':1})

The first thing we do is drop the bad bands (those in the atmospheric water absorption zones)

In [ ]:
ds = ds.where(ds.good_wavelengths.compute()==1,drop=True)

### Quick plot

In [ ]:
#select rgb
rgb = ds.sel(wavelengths=[650, 560, 470], method='nearest')
#make colors nicer
rgb = gamma_adjust(rgb)

In [ ]:
rgb.hvplot.rgb(x='longitude', y='latitude', bands='wavelengths', aspect = 'equal', frame_width=400,rasterize=True)

## 5. Mix

![](https://media.tenor.com/adoVIoM4eo4AAAAC/keyboard-cat.gif)

We will now create synthetically mixed spectra. We start with a spatial data set that contains the labelled locations of pure endmembers for 4 classes: _Fynbos, Pine, Rock and Water_.   
First lets extract the spectra at the specified locations from the EMIT data.

In [ ]:
#load points
gdf = gpd.read_file("data/emit_unmix.gpkg")
#add x and y locations as df columns
gdf['Longitude'] = gdf.centroid.x
gdf['Latitude'] = gdf.centroid.y
#drops cols we dont need
df = pd.DataFrame(gdf).drop(['geometry','ID'],axis=1)
#add id col
df['ID'] = df.index

We use another function from our utils package `extract_points` to extract the spectra from the emit array and return the results as a df

In [ ]:
df = extract_points(df,ds)

We need to convert the string names of our endmembers to int

In [ ]:
#encode labels to int
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])

#we will need this later
class_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
#convert int64 to int using dict comprehension
class_mapping = {k: int(v) if isinstance(v, np.int64) else v for k, v in class_mapping.items()}
#write to file
json.dump(class_mapping, open('data/classes.json', 'w'))

#select only the cols we need
df = df[['ID','wavelengths','reflectance','Category']]

Convert from long to wide df format and drop rows with na in any column

In [ ]:
df = (df
      .pivot(index=['ID','Category'], columns='wavelengths', values='reflectance') #pivot long to wide
      .dropna() #drop rows with na
      .reset_index() #add index as cols
     )
df

We want to use different endmembers to create the mixtures used to train and test the model. Testing model performance on endmembers it has never seen is a more realisitic simulation of how the model will perform when applied to the real data (an entire emit scene).   
  
We therefore create a separate endmember library for training and testing.

In [ ]:
#split into train and val
df_tr, df_val = train_test_split(df, test_size=0.4, random_state=42)

Now we do the mixing using `mix_fast` from `utils` (it is not very fast), we can specify how many synthetic mixtures we want in the second argument.

In [ ]:
#create synthetic mixtures
#takes 5 min if 20000 for each
df_tr_mix = mix_fast(df_tr,20000)
df_val_mix = mix_fast(df_val,20000)

In [ ]:
#save for later
#df_tr_mix.to_csv('data/trmix.csv', index=False)
#df_val_mix.to_csv('data/valmix.csv', index=False)
#df_tr_mix = pd.read_csv('data/trmix.csv')
#df_val_mix = pd.read_csv('data/valmix.csv')

last bit of cleaning involves separating the labels and data

In [ ]:
#drop cols we dont need
df_tr_mix = df_tr_mix.drop(['Category','ID'], axis=1)
df_val_mix = df_val_mix.drop(['Category','ID'], axis=1)

#index of last wavelength col
idx = 243

#split into features and labels
X_val_mix = df_val_mix.iloc[:, :idx]

X_tr_mix = df_tr_mix.iloc[:, :idx]
#convert labels to %
y_val_mix = df_val_mix.iloc[:, idx:]*100
y_tr_mix = df_tr_mix.iloc[:, idx:]*100

Have a look at our data:

In [ ]:
#first the spectra:
X_tr_mix

In [ ]:
#second the lavbels:
y_tr_mix

## 6. Fit models

#### XGBoost
We will use XGBoost to fit models. XGBoost is a fast, efficient machine learning library that uses gradient boosting, a method where new models fix errors of prior ones. It's __very__ popular for machine learning on tabular data. It is very fast and usually achieves results superior to Random Forests.

We follow a pretty standard python ML workflow
- choose a method 
- define a parameter grid
- do a random search over these params for the best model using a 5 fold cross- validation
- keep the best model

In [ ]:
#data as np arrays
X_tr = np.array(X_tr_mix)
X_val = np.array(X_val_mix)
y_tr = np.array(y_tr_mix)
y_val = np.array(y_val_mix)

In [ ]:
# Initialize the model
xgb = XGBRegressor(tree_method='hist')

# Define parameter grid
param_dist = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [5, 10, 20]
}

# Run grid search
random_search = RandomizedSearchCV(xgb, param_dist, cv=5, n_iter=3)
random_search = random_search.fit(X_tr, y_tr)

#keep the best model
best_model_xgb = random_search.best_estimator_

In [ ]:
#save the best model
#best_model_xgb.save_model('models/best_xgb_model.json')

In [ ]:
#reload the best model
#best_model_xgb = XGBRegressor()
#best_model_xgb.load_model('models/best_xgb_model.json')

#### Inspect model performance

In [ ]:
# Predict on validation data
val_pred = best_model_xgb.predict(X_val)
val_pred = np.clip(val_pred,0,100)

# Predict on training data
tr_pred = best_model_xgb.predict(X_tr)
tr_pred = np.clip(tr_pred,0,100)

#### Plot results for training data

In [ ]:
#training samples
nsamples = tr_pred.shape[0]
noutputs = tr_pred.shape[1]

fig, axs = plt.subplots(noutputs, figsize=(10, 4*noutputs))

for i in range(noutputs):
    rmse = np.sqrt(mean_squared_error(y_tr[:, i], tr_pred[:, i]))
    key = list(class_mapping.keys())[list(class_mapping.values()).index(i)]
    axs[i].scatter(y_tr[:, i],tr_pred[:, i],alpha=0.2)
    axs[i].set_title(f'Training class: {key}, RMSE: {rmse:.2f}')

plt.tight_layout()
plt.show()

#### Plot results for testing data

In [ ]:
#validation samples
nsamples = val_pred.shape[0]
noutputs = val_pred.shape[1]

fig, axs = plt.subplots(noutputs, figsize=(10, 4*noutputs))

for i in range(noutputs):
    rmse = np.sqrt(mean_squared_error(y_val[:, i], val_pred[:, i]))
    key = list(class_mapping.keys())[list(class_mapping.values()).index(i)]
    axs[i].scatter(y_val[:, i],val_pred[:, i],alpha=0.2)
    axs[i].set_title(f'Validation class: {key}, RMSE: {rmse:.2f}')

plt.tight_layout()
plt.show()

## Bonus: another ML method

[ROCKET](https://github.com/angus924/rocket) is a method for time series classification, making it specifically suitable for 1D data with temporal (or spectral) ordering. It works by generating a large number of random convolutional kernels which are applied to an input time series to produce features which are used with a linear classifier. It may be a better choice for this data as XGBoost is a method designed for tabluar data - it does not account for the ordering of features and thus is agnostic to the spectral postion. You can try to fit rocket models below, but beware, they can take longer than xgboost models to fit

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sktime.regression.kernel_based import RocketRegressor

In [ ]:
#params to search over
param_dist = {
    'estimator__num_kernels': [100, 500, 2000], 
}

# Define the estimator
reg = MultiOutputRegressor(RocketRegressor(rocket_transform='minirocket'))

# Perform the random search
random_search = RandomizedSearchCV(reg, param_dist, cv=5, n_iter=3)
random_search.fit(X_tr,y_tr)
best_model = random_search.best_estimator_

In [ ]:
#with open('models/rocketmodel.pkl', 'wb') as f:
#    pickle.dump(best_model, f)

In [ ]:
#with open('models/rocketmodel.pkl', 'rb') as f:
#    best_model = pickle.load(f)

In [ ]:
# Predict on validation data
val_pred = best_model.predict(X_val)
val_pred = np.clip(val_pred,0,100)

# Predict on training data
tr_pred = best_model.predict(X_tr)
tr_pred = np.clip(tr_pred,0,100)

In [ ]:
#training samples
nsamples = tr_pred.shape[0]
noutputs = tr_pred.shape[1]

fig, axs = plt.subplots(noutputs, figsize=(10, 4*noutputs))

for i in range(noutputs):
    rmse = np.sqrt(mean_squared_error(y_tr[:, i], tr_pred[:, i]))
    key = list(class_mapping.keys())[list(class_mapping.values()).index(i)]
    axs[i].scatter(y_tr[:, i],tr_pred[:, i],alpha=0.2)
    axs[i].set_title(f'Training class: {key}, RMSE: {rmse:.2f}')

plt.tight_layout()
plt.show()

In [ ]:
#validation samples
nsamples = val_pred.shape[0]
noutputs = val_pred.shape[1]

fig, axs = plt.subplots(noutputs, figsize=(10, 4*noutputs))

for i in range(noutputs):
    rmse = np.sqrt(mean_squared_error(y_val[:, i], val_pred[:, i]))
    key = list(class_mapping.keys())[list(class_mapping.values()).index(i)]
    axs[i].scatter(y_val[:, i],val_pred[:, i],alpha=0.2)
    axs[i].set_title(f'Validation class: {key}, RMSE: {rmse:.2f}')

plt.tight_layout()
plt.show()

## credits:

This lesson has borrowed from:    

[the EMIT-Data-Resources repository by LPDAAC/ the EMIT team](https://github.com/nasa/EMIT-Data-Resources) 

[Okunjeni et al 2013 RSE for the original methodology](https://www.sciencedirect.com/science/article/abs/pii/S0034425713002009)
